In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os

# to see in class example refer to 21/2/10

# Read the CSV

In [2]:
df = pd.read_csv("gentrified_LA_data01.csv")
# Drop the null columns where all values are null
#df = df.dropna(axis='columns', how='all')
# Drop the null rows
#df = df.dropna()
df.head()

OBJECTID  Zip Code  Rank                             Neighborhood  \
0         1     90001    37                                 Florence   
1         2     90002    57                      Watts/Green Meadows   
2         3     90003    33             Florence/Broadway-Manchester   
3         4     90004    15  East Hollywood/Larchmont/Windsor Square   
4         5     90005    20                   Koreatown/Mid-Wilshire   

   % Change in IRS Ratio  % Change in Median Household Size  \
0                -47.007                             -2.759   
1                -44.965                              1.683   
2                -41.524                              3.000   
3                -32.379                             -6.873   
4                -41.123                             -8.214   

   Change in % White Non Hispanic  Change in % >=25 with Bachelor's Degrees  \
0                          -0.086                                     1.561   
1                           0.346                                     1.309   
2                          -0.182                                     2.798   
3                           3.060                                    11.296   
4                           0.934                                     7.120   

   % Change in Rent (Inflation Adjusted)  Index Score  ...  \
0                                 24.858     0.389561  ...   
1                                 25.733     0.359525  ...   
2                                 35.303     0.402008  ...   
3                                 29.716     0.483000  ...   
4                                 24.864     0.460000  ...   

  MF SF Sold Demo 00-14  MF Total Acres Sold Demo 00-14  \
0               65653.0                          3.8364   
1                   NaN                             NaN   
2               31200.0                          0.3880   
3              440269.0                          6.8533   
4              223137.0                          7.0523   

   MF Sold Demo Average Age  Input_#5_Zip  RET Prop Sold Demoed 01-14  \
0                 94.181818       90001.0                         2.0   
1                       NaN       90002.0                         6.0   
2                 63.000000       90003.0                         5.0   
3                 66.307692       90004.0                         3.0   
4                 82.968750       90005.0                         4.0   

   RET SF Sold Demoed 01-14  RET Avg Age Sold Demoed 01-14  \
0                    3143.0                      36.000000   
1                   39111.0                      72.666667   
2                   45526.0                      72.600000   
3                   20474.0                      90.666667   
4                  114268.0                      66.250000   

   RET Acres Sold Demoed 01-14  Input_#6_Zip  2014_Median_HomePrice  
0                       1.0386       90001.0            248958.3333  
1                       1.4631       90002.0            237333.3333  
2                       2.3245       90003.0            271954.5833  
3                       1.0935       90004.0            853666.2500  
4                       2.6023       90005.0            550625.0000  

[5 rows x 69 columns]

# Select control model features, place into dataframe

In [3]:
# Set up x values/features df with just control model columns, this is cut #1

# show all columns
pd.set_option('display.max_columns', None)

# reduce set to cols 0-29
selected_features = df.iloc[:, 0:29]
selected_features.head()

OBJECTID  Zip Code  Rank                             Neighborhood  \
0         1     90001    37                                 Florence   
1         2     90002    57                      Watts/Green Meadows   
2         3     90003    33             Florence/Broadway-Manchester   
3         4     90004    15  East Hollywood/Larchmont/Windsor Square   
4         5     90005    20                   Koreatown/Mid-Wilshire   

   % Change in IRS Ratio  % Change in Median Household Size  \
0                -47.007                             -2.759   
1                -44.965                              1.683   
2                -41.524                              3.000   
3                -32.379                             -6.873   
4                -41.123                             -8.214   

   Change in % White Non Hispanic  Change in % >=25 with Bachelor's Degrees  \
0                          -0.086                                     1.561   
1                           0.346                                     1.309   
2                          -0.182                                     2.798   
3                           3.060                                    11.296   
4                           0.934                                     7.120   

   % Change in Rent (Inflation Adjusted)  Index Score Index Percentile  \
0                                 24.858     0.389561            0.669   
1                                 25.733     0.359525            0.486   
2                                 35.303     0.402008            0.706   
3                                 29.716     0.483000            0.871   
4                                 24.864     0.460000            0.825   

   Index Quartile  High to Low AGI Ratio (2000)  High to Low AGI Ratio (2014)  \
0               3                        24.339                        12.898   
1               2                        27.413                        15.087   
2               3                        31.808                        18.600   
3               4                         2.338                         1.581   
4               4                         4.074                         2.399   

   Median Household Income (2014)  Median Household Size (2000)  \
0                           34050                          4.35   
1                           30214                          4.16   
2                           30016                          4.00   
3                           38493                          2.91   
4                           31214                          2.80   

   Median Household Size (2013)  % White Non-Hispanic (2000)  \
0                          4.23                         0.78   
1                          4.23                         0.47   
2                          4.12                         0.68   
3                          2.71                        14.30   
4                          2.57                         7.56   

   % White Non-Hispanic (2014)  % >=25 w Bachelor's Degree (2000)  \
0                         0.69                               2.41   
1                         0.81                               3.00   
2                         0.49                               3.30   
3                        17.36                              22.30   
4                         8.49                              19.03   

   % >=25 w Bachelor's Degree (2014)  \
0                               3.97   
1                               4.31   
2                               6.09   
3                              33.59   
4                              26.15   

   Median Gross Rent (2000 - Inflation Adjusted)  Median Gross Rent 2014  \
0                                            781                     975   
1                                            770                     968   
2                                            786                    1064   
3                                   

In [4]:
# get the shape of the file
selected_features.shape

(110, 29)

In [5]:
# remove non-feature columns
selected_features = selected_features.drop(['OBJECTID', 'Zip Code', 'Rank', 'Neighborhood', 'Index Score', 'Index Percentile'], axis=1)
selected_features.head()

% Change in IRS Ratio  % Change in Median Household Size  \
0                -47.007                             -2.759   
1                -44.965                              1.683   
2                -41.524                              3.000   
3                -32.379                             -6.873   
4                -41.123                             -8.214   

   Change in % White Non Hispanic  Change in % >=25 with Bachelor's Degrees  \
0                          -0.086                                     1.561   
1                           0.346                                     1.309   
2                          -0.182                                     2.798   
3                           3.060                                    11.296   
4                           0.934                                     7.120   

   % Change in Rent (Inflation Adjusted)  Index Quartile  \
0                                 24.858               3   
1                                 25.733               2   
2                                 35.303               3   
3                                 29.716               4   
4                                 24.864               4   

   High to Low AGI Ratio (2000)  High to Low AGI Ratio (2014)  \
0                        24.339                        12.898   
1                        27.413                        15.087   
2                        31.808                        18.600   
3                         2.338                         1.581   
4                         4.074                         2.399   

   Median Household Income (2014)  Median Household Size (2000)  \
0                           34050                          4.35   
1                           30214                          4.16   
2                           30016                          4.00   
3                           38493                          2.91   
4                           31214                          2.80   

   Median Household Size (2013)  % White Non-Hispanic (2000)  \
0                          4.23                         0.78   
1                          4.23                         0.47   
2                          4.12                         0.68   
3                          2.71                        14.30   
4                          2.57                         7.56   

   % White Non-Hispanic (2014)  % >=25 w Bachelor's Degree (2000)  \
0                         0.69                               2.41   
1                         0.81                               3.00   
2                         0.49                               3.30   
3                        17.36                              22.30   
4                         8.49                              19.03   

   % >=25 w Bachelor's Degree (2014)  \
0                               3.97   
1                               4.31   
2                               6.09   
3                              33.59   
4                              26.15   

   Median Gross Rent (2000 - Inflation Adjusted)  Median Gross Rent 2014  \
0                                            781                     975   
1                                            770                     968   
2                                            786                    1064   
3                                            825                    1070   
4                                            752                     939   

   % Change in Median Gross Rent  \
0                       0.248399   
1                       0.257143   
2                       0.353690   
3                       0.296970   
4                       0.248670   

   Median Household Income (2000 - Inflation Adjusted)  \
0                                        33279.05739     
1                                        31443.73944     
2                                        30720.61042     
3                                        37931.

In [6]:
# get the shape of the file
selected_features.shape

(110, 23)

In [7]:
# get descriptive stats for each column
selected_features.describe()

% Change in IRS Ratio  % Change in Median Household Size  \
count             110.000000                         110.000000   
mean              -21.264636                           0.020573   
std                22.867960                           5.674417   
min               -72.601000                         -18.627000   
25%               -41.002500                          -3.422750   
50%               -20.856500                           1.056500   
75%                -3.971750                           4.541750   
max                47.208000                           9.137000   

       Change in % White Non Hispanic  \
count                      110.000000   
mean                        -1.951336   
std                          6.281333   
min                        -24.633000   
25%                         -5.554250   
50%                         -1.493500   
75%                          1.266750   
max                         26.815000   

       Change in % >=25 with Bachelor's Degrees  \
count                                110.000000   
mean                                   6.903400   
std                                    4.853364   
min                                   -0.313000   
25%                                    3.520000   
50%                                    5.998000   
75%                                   10.106250   
max                                   29.511000   

       % Change in Rent (Inflation Adjusted)  Index Quartile  \
count                             110.000000       110.00000   
mean                               27.477300         2.50000   
std                                16.148539         1.13129   
min                               -13.971000         1.00000   
25%                                21.481250         1.25000   
50%                                26.322500         2.50000   
75%                                31.071750         3.75000   
max                               108.064000         4.00000   

       High to Low AGI Ratio (2000)  High to Low AGI Ratio (2014)  \
count                    110.000000                    110.000000   
mean                       3.921400                      2.196400   
std                        7.059353                      3.563287   
min                        0.041000                      0.044000   
25%                        0.305000                      0.281500   
50%                        0.985000                      0.847000   
75%                        3.550250                      2.059750   
max                       37.713000                     18.600000   

       Median Household Income (2014)  Median Household Size (2000)  \
count                      110.000000                    110.000000   
mean                     58319.063636                      2.746091   
std                      28782.357449                      0.782085   
min                      12748.000000                      0.000000   
25%                      37353.250000                      2.250000   
50%                      52115.000000                      2.775000   
75%                      73258.000000                      3.085000   
max                     172443.000000                      4.650000   

       Median Household Size (2013)  % White Non-Hispanic (2000)  \
count                    110.000000                   110.000000   
mean                       2.757182                    36.371182   
std                        0.729768                    29.396129   
min                        1.270000                     0.000000   
25%                        2.267500                     8.885000   
50%                        2.690000                    29.900000   
75%                        3.115000                    64.552500   
max                        4.590000                    88.540000   

       % White Non-Hispanic (2014)  % >=25 w Bachelor's Degree (2000)  \
count                   110.000000              

### Assign x and y variables

In [8]:
# one-hot encode just the index quartile column
selected_features = pd.get_dummies(selected_features, prefix=['Index Quartile'], columns=['Index Quartile'])
selected_features.head()

% Change in IRS Ratio  % Change in Median Household Size  \
0                -47.007                             -2.759   
1                -44.965                              1.683   
2                -41.524                              3.000   
3                -32.379                             -6.873   
4                -41.123                             -8.214   

   Change in % White Non Hispanic  Change in % >=25 with Bachelor's Degrees  \
0                          -0.086                                     1.561   
1                           0.346                                     1.309   
2                          -0.182                                     2.798   
3                           3.060                                    11.296   
4                           0.934                                     7.120   

   % Change in Rent (Inflation Adjusted)  High to Low AGI Ratio (2000)  \
0                                 24.858                        24.339   
1                                 25.733                        27.413   
2                                 35.303                        31.808   
3                                 29.716                         2.338   
4                                 24.864                         4.074   

   High to Low AGI Ratio (2014)  Median Household Income (2014)  \
0                        12.898                           34050   
1                        15.087                           30214   
2                        18.600                           30016   
3                         1.581                           38493   
4                         2.399                           31214   

   Median Household Size (2000)  Median Household Size (2013)  \
0                          4.35                          4.23   
1                          4.16                          4.23   
2                          4.00                          4.12   
3                          2.91                          2.71   
4                          2.80                          2.57   

   % White Non-Hispanic (2000)  % White Non-Hispanic (2014)  \
0                         0.78                         0.69   
1                         0.47                         0.81   
2                         0.68                         0.49   
3                        14.30                        17.36   
4                         7.56                         8.49   

   % >=25 w Bachelor's Degree (2000)  % >=25 w Bachelor's Degree (2014)  \
0                               2.41                               3.97   
1                               3.00                               4.31   
2                               3.30                               6.09   
3                              22.30                              33.59   
4                              19.03                              26.15   

   Median Gross Rent (2000 - Inflation Adjusted)  Median Gross Rent 2014  \
0                                            781                     975   
1                                            770                     968   
2                                            786                    1064   
3                                            825                    1070   
4                                            752                     939   

   % Change in Median Gross Rent  \
0                       0.248399   
1                       0.257143   
2                       0.353690   
3                       0.296970   
4                       0.248670   

   Median Household Income (2000 - Inflation Adjusted)  \
0                                        33279.05739     
1                                        31443.73944     
2                                        30720.61042     
3                                        37931.27907     
4                                        30242.19046     

   % Change in Median Houshold Income (Inflation Adjusted

In [9]:
# get the shape of the file
selected_features.shape

(110, 26)

In [10]:
# Run the model using Index_Quartile_1 for the dependent variable (this will be removed later) and remove
# cols Index2, 3 and 4
selected_features = selected_features.iloc[:, 0:23]
selected_features.head()

% Change in IRS Ratio  % Change in Median Household Size  \
0                -47.007                             -2.759   
1                -44.965                              1.683   
2                -41.524                              3.000   
3                -32.379                             -6.873   
4                -41.123                             -8.214   

   Change in % White Non Hispanic  Change in % >=25 with Bachelor's Degrees  \
0                          -0.086                                     1.561   
1                           0.346                                     1.309   
2                          -0.182                                     2.798   
3                           3.060                                    11.296   
4                           0.934                                     7.120   

   % Change in Rent (Inflation Adjusted)  High to Low AGI Ratio (2000)  \
0                                 24.858                        24.339   
1                                 25.733                        27.413   
2                                 35.303                        31.808   
3                                 29.716                         2.338   
4                                 24.864                         4.074   

   High to Low AGI Ratio (2014)  Median Household Income (2014)  \
0                        12.898                           34050   
1                        15.087                           30214   
2                        18.600                           30016   
3                         1.581                           38493   
4                         2.399                           31214   

   Median Household Size (2000)  Median Household Size (2013)  \
0                          4.35                          4.23   
1                          4.16                          4.23   
2                          4.00                          4.12   
3                          2.91                          2.71   
4                          2.80                          2.57   

   % White Non-Hispanic (2000)  % White Non-Hispanic (2014)  \
0                         0.78                         0.69   
1                         0.47                         0.81   
2                         0.68                         0.49   
3                        14.30                        17.36   
4                         7.56                         8.49   

   % >=25 w Bachelor's Degree (2000)  % >=25 w Bachelor's Degree (2014)  \
0                               2.41                               3.97   
1                               3.00                               4.31   
2                               3.30                               6.09   
3                              22.30                              33.59   
4                              19.03                              26.15   

   Median Gross Rent (2000 - Inflation Adjusted)  Median Gross Rent 2014  \
0                                            781                     975   
1                                            770                     968   
2                                            786                    1064   
3                                            825                    1070   
4                                            752                     939   

   % Change in Median Gross Rent  \
0                       0.248399   
1                       0.257143   
2                       0.353690   
3                       0.296970   
4                       0.248670   

   Median Household Income (2000 - Inflation Adjusted)  \
0                                        33279.05739     
1                                        31443.73944     
2                                        30720.61042     
3                                        37931.27907     
4                                        30242.19046     

   % Change in Median Houshold Income (Inflation Adjusted

In [11]:
# get the shape of the file
selected_features.shape

(110, 23)

In [12]:
# put the target (what you want to classify) into a variable
target = selected_features["Index Quartile_1"]
target_names = ["lower gentrification", "high gentrification"]

In [13]:
# remove the target from the X variables
data = selected_features.drop("Index Quartile_1", axis=1)

# assign the column names to features so we assess importance later on
feature_names = data.columns

# view table
data.head()

% Change in IRS Ratio  % Change in Median Household Size  \
0                -47.007                             -2.759   
1                -44.965                              1.683   
2                -41.524                              3.000   
3                -32.379                             -6.873   
4                -41.123                             -8.214   

   Change in % White Non Hispanic  Change in % >=25 with Bachelor's Degrees  \
0                          -0.086                                     1.561   
1                           0.346                                     1.309   
2                          -0.182                                     2.798   
3                           3.060                                    11.296   
4                           0.934                                     7.120   

   % Change in Rent (Inflation Adjusted)  High to Low AGI Ratio (2000)  \
0                                 24.858                        24.339   
1                                 25.733                        27.413   
2                                 35.303                        31.808   
3                                 29.716                         2.338   
4                                 24.864                         4.074   

   High to Low AGI Ratio (2014)  Median Household Income (2014)  \
0                        12.898                           34050   
1                        15.087                           30214   
2                        18.600                           30016   
3                         1.581                           38493   
4                         2.399                           31214   

   Median Household Size (2000)  Median Household Size (2013)  \
0                          4.35                          4.23   
1                          4.16                          4.23   
2                          4.00                          4.12   
3                          2.91                          2.71   
4                          2.80                          2.57   

   % White Non-Hispanic (2000)  % White Non-Hispanic (2014)  \
0                         0.78                         0.69   
1                         0.47                         0.81   
2                         0.68                         0.49   
3                        14.30                        17.36   
4                         7.56                         8.49   

   % >=25 w Bachelor's Degree (2000)  % >=25 w Bachelor's Degree (2014)  \
0                               2.41                               3.97   
1                               3.00                               4.31   
2                               3.30                               6.09   
3                              22.30                              33.59   
4                              19.03                              26.15   

   Median Gross Rent (2000 - Inflation Adjusted)  Median Gross Rent 2014  \
0                                            781                     975   
1                                            770                     968   
2                                            786                    1064   
3                                            825                    1070   
4                                            752                     939   

   % Change in Median Gross Rent  \
0                       0.248399   
1                       0.257143   
2                       0.353690   
3                       0.296970   
4                       0.248670   

   Median Household Income (2000 - Inflation Adjusted)  \
0                                        33279.05739     
1                                        31443.73944     
2                                        30720.61042     
3                                        37931.27907     
4                                        30242.19046     

   % Change in Median Houshold Income (Inflation Adjusted

In [14]:
# get the shape of the file
data.shape

(110, 22)

### Split the data into training and testing

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [17]:
# confirm size of training set (default is 75% of the dataframe, so .75 * 6991 = 5243)
len(X_train)

82

In [18]:
# confirm size of test set (the other 25%)
len(X_test)

28

### Set up support vector machine linear classifier

In [19]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
#model.fit(X_train, y_train) <-- this gets removed because we're running a grid search estimator

In [20]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 13, 26],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [21]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train, y_train)

C:\Users\JTS\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ...... C=1, gamma=0.0001, score=0.8928571428571429, total=   7.6s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.6s remaining:    0.0s


[CV] ...... C=1, gamma=0.0001, score=0.9259259259259259, total=   4.6s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   12.3s remaining:    0.0s


[CV] ...... C=1, gamma=0.0001, score=0.8888888888888888, total=   5.5s
[CV] C=1, gamma=0.001 ................................................
[CV] ....... C=1, gamma=0.001, score=0.8928571428571429, total=   7.8s
[CV] C=1, gamma=0.001 ................................................
[CV] ....... C=1, gamma=0.001, score=0.9259259259259259, total=   4.8s
[CV] C=1, gamma=0.001 ................................................
[CV] ....... C=1, gamma=0.001, score=0.8888888888888888, total=   5.7s
[CV] C=1, gamma=0.01 .................................................
[CV] ........ C=1, gamma=0.01, score=0.8928571428571429, total=   7.7s
[CV] C=1, gamma=0.01 .................................................
[CV] ........ C=1, gamma=0.01, score=0.9259259259259259, total=   4.5s
[CV] C=1, gamma=0.01 .................................................
[CV] ........ C=1, gamma=0.01, score=0.8888888888888888, total=   5.5s
[CV] C=13, gamma=0.0001 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  3.1min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1, 13, 26], 'gamma': [0.0001, 0.001, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [22]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 1, 'gamma': 0.0001}


In [23]:
# List the best score
print(grid.best_score_)

0.9024390243902439


In [24]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [26]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["lower gentrification", "high gentrification"]))



                      precision    recall  f1-score   support

lower gentrification       0.90      0.90      0.90        21
 high gentrification       0.71      0.71      0.71         7

           micro avg       0.86      0.86      0.86        28
           macro avg       0.81      0.81      0.81        28
        weighted avg       0.86      0.86      0.86        28



# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# import joblib
# filename = 'your_name.sav'
# joblib.dump(your_model, filename)